# Installing Dependency

In [ ]:
%pip install google-play-scraper --upgrade
%pip install emoji --upgrade
%pip install Sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=aa390acda2d59cdce1517208437eb5cf10b3ed8ee12f9daad4d4120aa5d7eca4
  Stored in directory: /root/.cache/pip/wheels/86/62/9e/a6b27a681abcde69970dbc0326ff51955f3beac72f15696984
Successfully built emoji
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 KB 6.2 MB/s eta 0:00:00


# Import Dependency

In [ ]:
import pandas as pd
import numpy as np
# from google.colab import drive
from google_play_scraper import Sort, reviews, reviews_all
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import string
import re
import emoji
import ast
import json
import io
from collections import Counter

# Sastrawi
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
stemmer_factory = StemmerFactory()
stopword_factory = StopWordRemoverFactory()

stemmer = stemmer_factory.create_stemmer()
stopword_remover = stopword_factory.create_stop_word_remover()

In [ ]:
drive.mount('/content/gdrive')
gdrive_path = '/content/gdrive/My Drive/Kuliah/Skripsi/dataset/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Collect Sentiment

In [ ]:
gojek_app_id = 'com.gojek.app'
grab_app_id = 'com.grabtaxi.passenger'
maxim_app_id = 'com.taxsee.taxsee'

In [ ]:
def iterate_reviews_till_empty(review, continuation_token, dataset_name, appId):
    while continuation_token.token != None:
      csv_path = '/content/gdrive/MyDrive/Kuliah/Skripsi/dataset/{}.csv'.format(dataset_name)

      old_df = pd.read_csv(csv_path, quoting=csv.QUOTE_ALL, index_col='reviewId')
      
      df = pd.DataFrame(review, columns=['review'])
      df = df.join(pd.DataFrame(df.pop('review').tolist()))

      df.pop('userName')
      df.pop('userImage')
      df.pop('thumbsUpCount')
      df.pop('reviewCreatedVersion')
      df.pop('at')
      df.pop('replyContent')
      df.pop('repliedAt')

      df = df.drop_duplicates(subset='content', keep=False)
      df = df.drop_duplicates(subset='reviewId', keep=False)
      df = df.reset_index(drop=True)
      df = df.set_index('reviewId')
      
      df = old_df.append(df)
      df.to_csv(csv_path, quoting=csv.QUOTE_ALL)
      print(df.shape)

      review, continuation_token = reviews(
          appId,
          continuation_token=continuation_token
      )
      review = np.array(review)

def get_sentiment(appId, dataset_name, score=5):
    review, continuation_token = reviews(
        appId,
        lang='id', # defaults to 'en'
        country='id', # defaults to 'us'
        sort=Sort.NEWEST, # defaults to Sort.NEWEST
        count=200,
        filter_score_with=score
    )
    review = np.array(review)
    iterate_reviews_till_empty(review, continuation_token, dataset_name=dataset_name, appId=appId)

In [ ]:
get_sentiment(gojek_app_id, 'gojek', score=1)

In [ ]:
get_sentiment(grab_app_id, 'grab', score=1)

In [ ]:
get_sentiment(maxim_app_id, 'maxim', score=5)

In [ ]:
# workspace_dir = '/content/gdrive/MyDrive/Kuliah/Skripsi/dataset/'
workspace_dir = 'J:/My Drive/Kuliah/Skripsi/dataset/'
sentiment_gojek = '{}gojek.csv'.format(workspace_dir)
sentiment_grab = '{}grab.csv'.format(workspace_dir)
sentiment_maxim = '{}maxim.csv'.format(workspace_dir)

#Read Dataset

In [ ]:
gojek = pd.read_csv(sentiment_gojek, quoting=csv.QUOTE_ALL, index_col='reviewId')
grab = pd.read_csv(sentiment_grab, quoting=csv.QUOTE_ALL, index_col='reviewId')
maxim = pd.read_csv(sentiment_maxim, quoting=csv.QUOTE_ALL, index_col='reviewId')

print(gojek.shape)
gojek['score'].value_counts()

(89770, 8)


5.0    73201
1.0    16569
Name: score, dtype: int64

In [ ]:
print(grab.shape)
grab['score'].value_counts()

(310829, 2)


5.0    215289
1.0     95540
Name: score, dtype: int64

In [ ]:
print(maxim.shape)
maxim['score'].value_counts()

(148311, 2)


5.0    129348
1.0     18963
Name: score, dtype: int64

In [ ]:
slang_dataset = '{}colloquial_indonesian_lexicon.csv'.format(workspace_dir)
slang_dictionary = pd.read_csv(slang_dataset)

slang_dictionary = dict(zip(slang_dictionary['slang'], slang_dictionary['formal']))
slang_dictionary

{'woww': 'wow',
 'aminn': 'amin',
 'met': 'selamat',
 'netaas': 'menetas',
 'keberpa': 'keberapa',
 'eeeehhhh': 'eh',
 'kata2nyaaa': 'kata-katanya',
 'hallo': 'halo',
 'kaka': 'kakak',
 'ka': 'kak',
 'daah': 'dah',
 'aaaaahhhh': 'ah',
 'yaa': 'ya',
 'smga': 'semoga',
 'slalu': 'selalu',
 'amiin': 'amin',
 'kk': 'kakak',
 'trus': 'terus',
 'sii': 'sih',
 'nyenengin': 'menyenangkan',
 'bgt': 'banget',
 'gemess': 'gemas',
 'akuuu': 'aku',
 'jgn': 'jangan',
 'udah': 'sudah',
 'gitu': 'begitu',
 'aja': 'saja',
 'gemesiin': 'menggemaskan',
 'menyenangkn': 'menyenangkan',
 'rb': 'ribu',
 'akau': 'aku',
 'saranin': 'menyarankan',
 'nemuin': 'menemukan',
 'yah': 'ya',
 'mnis2nya': 'manis-manisnya',
 'gtu': 'begitu',
 'tu': 'itu',
 'idolaa': 'idola',
 'bangett': 'banget',
 'kaakk': 'kak',
 'baaiikkk': 'baik',
 'telaatt': 'telat',
 'kaak': 'kak',
 'yg': 'yang',
 'lg': 'lagi',
 'skarang': 'sekarang',
 'sempet': 'sempat',
 'karna': 'karena',
 'ngga': 'enggak',
 'mantep': 'mantap',
 'loohhh': 'loh',

In [ ]:
def lowercase(text):
  return text.lower()

def remove_url(text):
  return re.sub(r'http\S+', '', text)

def remove_emoji(text):
  return emoji.replace_emoji(text)

def remove_number(text):
  exclusion_list = string.digits
  mapping_table = str.maketrans(exclusion_list, ' '*len(exclusion_list))
  return text.translate(mapping_table)

def remove_punctuation(text):
  exclusion_list = string.punctuation
  mapping_table = str.maketrans(exclusion_list, ' '*len(exclusion_list))
  return text.translate(mapping_table)

def remove_number_and_punctuation(text):
  number_removed = remove_number(text)
  punctuation_removed = remove_punctuation(number_removed)
  return punctuation_removed

def stripping(text):
  return text.strip()

def text_cleaning(text):
  lowercased = text.lower()
  url_removed = remove_url(lowercased)
  emoji_removed = remove_emoji(url_removed)
  number_punctuation_removed = remove_number_and_punctuation(emoji_removed)
  stripped = stripping(number_punctuation_removed)
  return stripped

def slang_handling(text):
  words = []
  for word in text.split():
    if word in slang_dictionary:
      words.append(slang_dictionary[word])
    else:
      words.append(word)
  return ' '.join(words)

def remove_stopword(text):
  stopwords = set(stopword_factory.get_stop_words())
  stopword_removed = [word for word in text.split() if word not in stopwords]
  return ' '.join(stopword_removed)

def stemming(text):
  return stemmer.stem(text)

def tokenizing(text):
  return text.split()

## Gojek

### Filter 10 Words Sentence

In [ ]:
gojek = pd.read_csv(sentiment_gojek, quoting=csv.QUOTE_ALL, index_col='reviewId')
grab = pd.read_csv(sentiment_grab, quoting=csv.QUOTE_ALL, index_col='reviewId')
maxim = pd.read_csv(sentiment_maxim, quoting=csv.QUOTE_ALL, index_col='reviewId')

print(gojek.shape)
gojek['score'].value_counts()

(89770, 8)


5.0    73201
1.0    16569
Name: score, dtype: int64

In [ ]:
gojek['lowercased'] = gojek['content'].map(lowercase)
gojek['cleaned'] = gojek['lowercased'].map(text_cleaning)
gojek['slang_handled'] = gojek['cleaned'].map(slang_handling)
gojek['stopword_removed'] = gojek['slang_handled'].map(remove_stopword)
gojek['stemmed'] = gojek['stopword_removed'].map(stemming)
gojek['tokenized'] = gojek['stemmed'].map(tokenizing)
gojek = gojek[['content', 'lowercased', 'cleaned', 'slang_handled', 'stopword_removed', 'stemmed', 'tokenized', 'score']]
gojek['word_count'] = gojek['cleaned'].map(lambda x: x.split())
gojek

,content,lowercased,cleaned,slang_handled,stopword_removed,stemmed,tokenized,score,word_count
reviewId,,,,,,,,,
dc6878fe-f176-487e-a9e4-3e1a92c4fd26,Maps nya kocak alamat gaada semua susah amat n...,maps nya kocak alamat gaada semua susah amat n...,maps nya kocak alamat gaada semua susah amat n...,maps nya kocak alamat enggak ada semua susah a...,maps kocak alamat susah nyarinya ubah rute gil...,maps kocak alamat susah nyarinya ubah rute gil...,"[maps, kocak, alamat, susah, nyarinya, ubah, r...",1.0,"[maps, nya, kocak, alamat, gaada, semua, susah..."
4e4a51bb-206e-4ea0-8b40-007f69043907,"Saat saya melakukan order food, pencarian drai...","saat saya melakukan order food, pencarian drai...",saat saya melakukan order food pencarian drai...,saat saya melakukan order food pencarian draiv...,order food pencarian draivernya ketemu orderan...,order food cari draivernya ketemu order makan ...,"[order, food, cari, draivernya, ketemu, order,...",1.0,"[saat, saya, melakukan, order, food, pencarian..."
18f99180-ee86-496b-995d-f3f65596bc2d,Baru pesan aja udah minta tarip kaya angkot pa...,baru pesan aja udah minta tarip kaya angkot pa...,baru pesan aja udah minta tarip kaya angkot pa...,baru pesan saja sudah meminta tarip kayak angk...,pesan tarip kayak angkot aplikasi uang segitu,pesan tarip kayak angkot aplikasi uang segitu,"[pesan, tarip, kayak, angkot, aplikasi, uang, ...",1.0,"[baru, pesan, aja, udah, minta, tarip, kaya, a..."
af56172a-cc7b-48b6-ae77-4112fb510484,"Bagusnya di awal doang, setelah lebih lama pak...","bagusnya di awal doang, setelah lebih lama pak...",bagusnya di awal doang setelah lebih lama pak...,bagusnya di awal doang setelah lebih lama paka...,bagusnya doang pakai susah banget driver pakai...,bagus doang pakai susah banget driver pakai fi...,"[bagus, doang, pakai, susah, banget, driver, p...",1.0,"[bagusnya, di, awal, doang, setelah, lebih, la..."
4eae708d-d8b7-4bd4-accb-2cfdfd369c90,Aplikasi ini buruk sekali saya tidak ada mesan...,aplikasi ini buruk sekali saya tidak ada mesan...,aplikasi ini buruk sekali saya tidak ada mesan...,aplikasi ini buruk sekali saya tidak ada mesan...,aplikasi buruk mesan daftar kode verifikasi te...,aplikasi buruk mesan daftar kode verifikasi te...,"[aplikasi, buruk, mesan, daftar, kode, verifik...",1.0,"[aplikasi, ini, buruk, sekali, saya, tidak, ad..."
...,...,...,...,...,...,...,...,...,...
ff2b390e-c3ef-4070-8577-2284098a86ed,kakaknya sangat ramah terus baik lagi,kakaknya sangat ramah terus baik lagi,kakaknya sangat ramah terus baik lagi,kakaknya sangat ramah terus baik lagi,kakaknya ramah,kakak ramah,"[kakak, ramah]",5.0,"[kakaknya, sangat, ramah, terus, baik, lagi]"
dc10c361-014d-4e30-b71e-de323beef663,Tolong diadakan promo biar makin ramai customerny,tolong diadakan promo biar makin ramai customerny,tolong diadakan promo biar makin ramai customerny,tolong diadakan promo biar makin ramai customerny,tolong diadakan promo biar ramai customerny,tolong ada promo biar ramai customerny,"[tolong, ada, promo, biar, ramai, customerny]",5.0,"[tolong, diadakan, promo, biar, makin, ramai, ..."
164a5c78-c3c3-4e56-881c-0e4778ef846c,Terima kasih sudah bantu angkat nenek saya,terima kasih sudah bantu angkat nenek saya,terima kasih sudah bantu angkat nenek saya,terima kasih sudah bantu angkat nenek saya,terima kasih bantu angkat nenek,terima kasih bantu angkat nenek,"[terima, kasih, bantu, angkat, nenek]",5.0,"[terima, kasih, sudah, bantu, angkat, nenek, s..."


In [ ]:
gojek = gojek[gojek['word_count'].str.len() >= 10]
gojek.to_csv('{}gojek_filter_10.csv'.format(workspace_dir), quoting=csv.QUOTE_ALL)

In [ ]:
gojek['word_count'].str.len().describe()

count    25491.000000
mean        21.619356
std         14.764427
min         10.000000
25%         12.000000
50%         16.000000
75%         25.000000
max        105.000000
Name: word_count, dtype: float64

In [ ]:
gojek['tokenized'].str.len().describe()

count    25491.000000
mean        11.656742
std          7.458452
min          1.000000
25%          7.000000
50%          9.000000
75%         14.000000
max         66.000000
Name: tokenized, dtype: float64

In [ ]:
print(gojek.shape)
gojek['score'].value_counts()

(25491, 9)


5.0    14331
1.0    11160
Name: score, dtype: int64

In [ ]:
new_gojek = gojek.copy(deep=True).head(1000)
new_gojek = new_gojek.append(gojek.copy(deep=True).tail(1000))
new_gojek['score'].value_counts()

C:\Users\onirutlA\AppData\Local\Temp\ipykernel_5140\2616523383.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_gojek = new_gojek.append(gojek.copy(deep=True).tail(1000))


1.0    1000
5.0    1000
Name: score, dtype: int64

In [ ]:
new_gojek.to_csv('{}gojek_1_5_2K_10.csv'.format(workspace_dir), quoting=csv.QUOTE_ALL)

### Filter 7 Words Sentence

In [ ]:
gojek = pd.read_csv(sentiment_gojek, quoting=csv.QUOTE_ALL, index_col='reviewId')
grab = pd.read_csv(sentiment_grab, quoting=csv.QUOTE_ALL, index_col='reviewId')
maxim = pd.read_csv(sentiment_maxim, quoting=csv.QUOTE_ALL, index_col='reviewId')

print(gojek.shape)
gojek['score'].value_counts()

(89770, 8)


5.0    73201
1.0    16569
Name: score, dtype: int64

In [ ]:
gojek['lowercased'] = gojek['content'].map(lowercase)
gojek['cleaned'] = gojek['lowercased'].map(text_cleaning)
gojek['slang_handled'] = gojek['cleaned'].map(slang_handling)
gojek['stopword_removed'] = gojek['slang_handled'].map(remove_stopword)
gojek['stemmed'] = gojek['stopword_removed'].map(stemming)
gojek['tokenized'] = gojek['stemmed'].map(tokenizing)
gojek = gojek[['content', 'lowercased', 'cleaned', 'slang_handled', 'stopword_removed', 'stemmed', 'tokenized', 'score']]
gojek['word_count'] = gojek['cleaned'].map(lambda x: x.split())
gojek

,content,lowercased,cleaned,slang_handled,stopword_removed,stemmed,tokenized,score,word_count
reviewId,,,,,,,,,
dc6878fe-f176-487e-a9e4-3e1a92c4fd26,Maps nya kocak alamat gaada semua susah amat n...,maps nya kocak alamat gaada semua susah amat n...,maps nya kocak alamat gaada semua susah amat n...,maps nya kocak alamat enggak ada semua susah a...,maps kocak alamat susah nyarinya ubah rute gil...,maps kocak alamat susah nyarinya ubah rute gil...,"[maps, kocak, alamat, susah, nyarinya, ubah, r...",1.0,"[maps, nya, kocak, alamat, gaada, semua, susah..."
4e4a51bb-206e-4ea0-8b40-007f69043907,"Saat saya melakukan order food, pencarian drai...","saat saya melakukan order food, pencarian drai...",saat saya melakukan order food pencarian drai...,saat saya melakukan order food pencarian draiv...,order food pencarian draivernya ketemu orderan...,order food cari draivernya ketemu order makan ...,"[order, food, cari, draivernya, ketemu, order,...",1.0,"[saat, saya, melakukan, order, food, pencarian..."
18f99180-ee86-496b-995d-f3f65596bc2d,Baru pesan aja udah minta tarip kaya angkot pa...,baru pesan aja udah minta tarip kaya angkot pa...,baru pesan aja udah minta tarip kaya angkot pa...,baru pesan saja sudah meminta tarip kayak angk...,pesan tarip kayak angkot aplikasi uang segitu,pesan tarip kayak angkot aplikasi uang segitu,"[pesan, tarip, kayak, angkot, aplikasi, uang, ...",1.0,"[baru, pesan, aja, udah, minta, tarip, kaya, a..."
af56172a-cc7b-48b6-ae77-4112fb510484,"Bagusnya di awal doang, setelah lebih lama pak...","bagusnya di awal doang, setelah lebih lama pak...",bagusnya di awal doang setelah lebih lama pak...,bagusnya di awal doang setelah lebih lama paka...,bagusnya doang pakai susah banget driver pakai...,bagus doang pakai susah banget driver pakai fi...,"[bagus, doang, pakai, susah, banget, driver, p...",1.0,"[bagusnya, di, awal, doang, setelah, lebih, la..."
4eae708d-d8b7-4bd4-accb-2cfdfd369c90,Aplikasi ini buruk sekali saya tidak ada mesan...,aplikasi ini buruk sekali saya tidak ada mesan...,aplikasi ini buruk sekali saya tidak ada mesan...,aplikasi ini buruk sekali saya tidak ada mesan...,aplikasi buruk mesan daftar kode verifikasi te...,aplikasi buruk mesan daftar kode verifikasi te...,"[aplikasi, buruk, mesan, daftar, kode, verifik...",1.0,"[aplikasi, ini, buruk, sekali, saya, tidak, ad..."
...,...,...,...,...,...,...,...,...,...
ff2b390e-c3ef-4070-8577-2284098a86ed,kakaknya sangat ramah terus baik lagi,kakaknya sangat ramah terus baik lagi,kakaknya sangat ramah terus baik lagi,kakaknya sangat ramah terus baik lagi,kakaknya ramah,kakak ramah,"[kakak, ramah]",5.0,"[kakaknya, sangat, ramah, terus, baik, lagi]"
dc10c361-014d-4e30-b71e-de323beef663,Tolong diadakan promo biar makin ramai customerny,tolong diadakan promo biar makin ramai customerny,tolong diadakan promo biar makin ramai customerny,tolong diadakan promo biar makin ramai customerny,tolong diadakan promo biar ramai customerny,tolong ada promo biar ramai customerny,"[tolong, ada, promo, biar, ramai, customerny]",5.0,"[tolong, diadakan, promo, biar, makin, ramai, ..."
164a5c78-c3c3-4e56-881c-0e4778ef846c,Terima kasih sudah bantu angkat nenek saya,terima kasih sudah bantu angkat nenek saya,terima kasih sudah bantu angkat nenek saya,terima kasih sudah bantu angkat nenek saya,terima kasih bantu angkat nenek,terima kasih bantu angkat nenek,"[terima, kasih, bantu, angkat, nenek]",5.0,"[terima, kasih, sudah, bantu, angkat, nenek, s..."


In [ ]:
gojek = gojek[gojek['word_count'].str.len() >= 7]
gojek.to_csv('{}gojek_filter_7.csv'.format(workspace_dir), quoting=csv.QUOTE_ALL)

In [ ]:
gojek['word_count'].str.len().describe()

count    39427.000000
mean        16.747457
std         13.586106
min          7.000000
25%          8.000000
50%         12.000000
75%         19.000000
max        105.000000
Name: word_count, dtype: float64

In [ ]:
gojek['tokenized'].str.len().describe()

count    39427.000000
mean         9.254420
std          6.869915
min          0.000000
25%          5.000000
50%          7.000000
75%         11.000000
max         66.000000
Name: tokenized, dtype: float64

In [ ]:
print(gojek.shape)
gojek['score'].value_counts()

(39427, 9)


5.0    26056
1.0    13371
Name: score, dtype: int64

In [ ]:
new_gojek = gojek.copy(deep=True).head(1000)
new_gojek = new_gojek.append(gojek.copy(deep=True).tail(1000))
new_gojek['score'].value_counts()

C:\Users\onirutlA\AppData\Local\Temp\ipykernel_5140\2616523383.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_gojek = new_gojek.append(gojek.copy(deep=True).tail(1000))


1.0    1000
5.0    1000
Name: score, dtype: int64

In [ ]:
new_gojek.to_csv('{}gojek_1_5_2K_7.csv'.format(workspace_dir), quoting=csv.QUOTE_ALL)

## Grab

### Filter 10 Words Sentence

In [ ]:
gojek = pd.read_csv(sentiment_gojek, quoting=csv.QUOTE_ALL, index_col='reviewId')
grab = pd.read_csv(sentiment_grab, quoting=csv.QUOTE_ALL, index_col='reviewId')
maxim = pd.read_csv(sentiment_maxim, quoting=csv.QUOTE_ALL, index_col='reviewId')

print(gojek.shape)
gojek['score'].value_counts()

(89770, 8)


5.0    73201
1.0    16569
Name: score, dtype: int64

In [ ]:
grab['content'] = grab['content'].astype(dtype='str')
grab['lowercased'] = grab['content'].map(lowercase)
grab['cleaned'] = grab['lowercased'].map(text_cleaning)
grab['word_count'] = grab['cleaned'].map(lambda x: x.split())
grab['slang_handled'] = grab['cleaned'].map(slang_handling)
grab['stopword_removed'] = grab['slang_handled'].map(remove_stopword)
grab['stemmed'] = grab['stopword_removed'].map(stemming)
grab['tokenized'] = grab['stemmed'].map(tokenizing)
grab['score'].value_counts()

5.0    215289
1.0     95540
Name: score, dtype: int64

In [ ]:
grab = grab[grab['word_count'].str.len() >= 10]
grab.to_csv('{}grab_filter_10.csv'.format(workspace_dir), quoting=csv.QUOTE_ALL)

In [ ]:
grab['word_count'].str.len().describe()

count    72931.000000
mean        22.932004
std         14.871430
min         10.000000
25%         13.000000
50%         18.000000
75%         28.000000
max        162.000000
Name: word_count, dtype: float64

In [ ]:
grab['tokenized'].str.len().describe()

count    72931.000000
mean        11.602158
std          7.730304
min          0.000000
25%          7.000000
50%          9.000000
75%         14.000000
max         79.000000
Name: tokenized, dtype: float64

In [ ]:
print(grab.shape)
grab['score'].value_counts()

(72931, 9)


1.0    51471
5.0    21460
Name: score, dtype: int64

In [ ]:
new_grab = grab.copy(deep=True).head(1000)
new_grab = new_grab.append(grab.copy(deep=True).tail(1000))
new_grab['score'].value_counts()

C:\Users\onirutlA\AppData\Local\Temp\ipykernel_5140\3574248348.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_grab = new_grab.append(grab.copy(deep=True).tail(1000))


5.0    1000
1.0    1000
Name: score, dtype: int64

In [ ]:
new_grab.to_csv('{}grab_1_5_2K_10.csv'.format(workspace_dir), quoting=csv.QUOTE_ALL)

### Filter 7 Words Sentence

In [ ]:
gojek = pd.read_csv(sentiment_gojek, quoting=csv.QUOTE_ALL, index_col='reviewId')
grab = pd.read_csv(sentiment_grab, quoting=csv.QUOTE_ALL, index_col='reviewId')
maxim = pd.read_csv(sentiment_maxim, quoting=csv.QUOTE_ALL, index_col='reviewId')

print(gojek.shape)
gojek['score'].value_counts()

(89770, 8)


5.0    73201
1.0    16569
Name: score, dtype: int64

In [ ]:
grab['content'] = grab['content'].astype(dtype='str')
grab['lowercased'] = grab['content'].map(lowercase)
grab['cleaned'] = grab['lowercased'].map(text_cleaning)
grab['word_count'] = grab['cleaned'].map(lambda x: x.split())
grab['slang_handled'] = grab['cleaned'].map(slang_handling)
grab['stopword_removed'] = grab['slang_handled'].map(remove_stopword)
grab['stemmed'] = grab['stopword_removed'].map(stemming)
grab['tokenized'] = grab['stemmed'].map(tokenizing)
grab['score'].value_counts()

5.0    215289
1.0     95540
Name: score, dtype: int64

In [ ]:
grab = grab[grab['word_count'].str.len() >= 7]
grab.to_csv('{}grab_filter_7.csv'.format(workspace_dir), quoting=csv.QUOTE_ALL)

In [ ]:
grab['word_count'].str.len().describe()

count    100692.000000
mean         18.782326
std          14.338929
min           7.000000
25%           9.000000
50%          14.000000
75%          23.000000
max         162.000000
Name: word_count, dtype: float64

In [ ]:
grab['tokenized'].str.len().describe()

count    100692.000000
mean          9.605351
std           7.370776
min           0.000000
25%           5.000000
50%           7.000000
75%          12.000000
max          79.000000
Name: tokenized, dtype: float64

In [ ]:
print(grab.shape)
grab['score'].value_counts()

(100692, 9)


1.0    63346
5.0    37346
Name: score, dtype: int64

In [ ]:
new_grab = grab.copy(deep=True).head(1000)
new_grab = new_grab.append(grab.copy(deep=True).tail(1000))
new_grab['score'].value_counts()

C:\Users\onirutlA\AppData\Local\Temp\ipykernel_5140\3574248348.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_grab = new_grab.append(grab.copy(deep=True).tail(1000))


5.0    1000
1.0    1000
Name: score, dtype: int64

In [ ]:
new_grab.to_csv('{}grab_1_5_2K_7.csv'.format(workspace_dir), quoting=csv.QUOTE_ALL)

## Maxim

### Filter 10 Words Sentence

In [ ]:
gojek = pd.read_csv(sentiment_gojek, quoting=csv.QUOTE_ALL, index_col='reviewId')
grab = pd.read_csv(sentiment_grab, quoting=csv.QUOTE_ALL, index_col='reviewId')
maxim = pd.read_csv(sentiment_maxim, quoting=csv.QUOTE_ALL, index_col='reviewId')

print(gojek.shape)
gojek['score'].value_counts()

(89770, 8)


5.0    73201
1.0    16569
Name: score, dtype: int64

In [ ]:
maxim['lowercased'] = maxim['content'].map(lowercase)
maxim['cleaned'] = maxim['lowercased'].map(text_cleaning)
maxim['slang_handled'] = maxim['cleaned'].map(slang_handling)
maxim['stopword_removed'] = maxim['slang_handled'].map(remove_stopword)
maxim['stemmed'] = maxim['stopword_removed'].map(stemming)
maxim['tokenized'] = maxim['stemmed'].map(tokenizing)
maxim = maxim[['content', 'lowercased', 'cleaned', 'slang_handled', 'stopword_removed', 'stemmed', 'tokenized', 'score']]
maxim['word_count'] = maxim['cleaned'].map(lambda x: x.split())
maxim

,content,lowercased,cleaned,slang_handled,stopword_removed,stemmed,tokenized,score,word_count
reviewId,,,,,,,,,
dc6878fe-f176-487e-a9e4-3e1a92c4fd26,Maps nya kocak alamat gaada semua susah amat n...,maps nya kocak alamat gaada semua susah amat n...,maps nya kocak alamat gaada semua susah amat n...,maps nya kocak alamat enggak ada semua susah a...,maps kocak alamat susah nyarinya ubah rute gil...,maps kocak alamat susah nyarinya ubah rute gil...,"[maps, kocak, alamat, susah, nyarinya, ubah, r...",1.0,"[maps, nya, kocak, alamat, gaada, semua, susah..."
4e4a51bb-206e-4ea0-8b40-007f69043907,"Saat saya melakukan order food, pencarian drai...","saat saya melakukan order food, pencarian drai...",saat saya melakukan order food pencarian drai...,saat saya melakukan order food pencarian draiv...,order food pencarian draivernya ketemu orderan...,order food cari draivernya ketemu order makan ...,"[order, food, cari, draivernya, ketemu, order,...",1.0,"[saat, saya, melakukan, order, food, pencarian..."
18f99180-ee86-496b-995d-f3f65596bc2d,Baru pesan aja udah minta tarip kaya angkot pa...,baru pesan aja udah minta tarip kaya angkot pa...,baru pesan aja udah minta tarip kaya angkot pa...,baru pesan saja sudah meminta tarip kayak angk...,pesan tarip kayak angkot aplikasi uang segitu,pesan tarip kayak angkot aplikasi uang segitu,"[pesan, tarip, kayak, angkot, aplikasi, uang, ...",1.0,"[baru, pesan, aja, udah, minta, tarip, kaya, a..."
af56172a-cc7b-48b6-ae77-4112fb510484,"Bagusnya di awal doang, setelah lebih lama pak...","bagusnya di awal doang, setelah lebih lama pak...",bagusnya di awal doang setelah lebih lama pak...,bagusnya di awal doang setelah lebih lama paka...,bagusnya doang pakai susah banget driver pakai...,bagus doang pakai susah banget driver pakai fi...,"[bagus, doang, pakai, susah, banget, driver, p...",1.0,"[bagusnya, di, awal, doang, setelah, lebih, la..."
4eae708d-d8b7-4bd4-accb-2cfdfd369c90,Aplikasi ini buruk sekali saya tidak ada mesan...,aplikasi ini buruk sekali saya tidak ada mesan...,aplikasi ini buruk sekali saya tidak ada mesan...,aplikasi ini buruk sekali saya tidak ada mesan...,aplikasi buruk mesan daftar kode verifikasi te...,aplikasi buruk mesan daftar kode verifikasi te...,"[aplikasi, buruk, mesan, daftar, kode, verifik...",1.0,"[aplikasi, ini, buruk, sekali, saya, tidak, ad..."
...,...,...,...,...,...,...,...,...,...
dc10c361-014d-4e30-b71e-de323beef663,Tolong diadakan promo biar makin ramai customerny,tolong diadakan promo biar makin ramai customerny,tolong diadakan promo biar makin ramai customerny,tolong diadakan promo biar makin ramai customerny,tolong diadakan promo biar ramai customerny,tolong ada promo biar ramai customerny,"[tolong, ada, promo, biar, ramai, customerny]",5.0,"[tolong, diadakan, promo, biar, makin, ramai, ..."
164a5c78-c3c3-4e56-881c-0e4778ef846c,Terima kasih sudah bantu angkat nenek saya,terima kasih sudah bantu angkat nenek saya,terima kasih sudah bantu angkat nenek saya,terima kasih sudah bantu angkat nenek saya,terima kasih bantu angkat nenek,terima kasih bantu angkat nenek,"[terima, kasih, bantu, angkat, nenek]",5.0,"[terima, kasih, sudah, bantu, angkat, nenek, s..."
5005c096-6dc0-461d-a2d4-33dc40e30b27,Tks semoga makin banyak armada dan pelayanan t...,tks semoga makin banyak armada dan pelayanan t...,tks semoga makin banyak armada dan pelayanan t...,tks semoga makin banyak armada dan pelayanan t...,tks semoga armada pelayanan bagus,tks moga armada layan bagus,"[tks, moga, armada, layan, bagus]",5.0,"[tks, semoga, makin, banyak, armada, dan, pela..."


In [ ]:
maxim = maxim[maxim['word_count'].str.len() >= 10]
maxim.to_csv('{}maxim_filter_10.csv'.format(workspace_dir), quoting=csv.QUOTE_ALL)

In [ ]:
maxim['word_count'].str.len().describe()

count    25496.000000
mean        21.618058
std         14.763376
min         10.000000
25%         12.000000
50%         16.000000
75%         25.000000
max        105.000000
Name: word_count, dtype: float64

In [ ]:
maxim['tokenized'].str.len().describe()

count    25496.000000
mean        11.654495
std          7.459448
min          0.000000
25%          7.000000
50%          9.000000
75%         14.000000
max         66.000000
Name: tokenized, dtype: float64

In [ ]:
print(maxim.shape)
maxim['score'].value_counts()

(25496, 9)


5.0    14335
1.0    11161
Name: score, dtype: int64

In [ ]:
new_maxim = maxim.copy(deep=True).head(1000)
new_maxim = new_maxim.append(maxim.copy(deep=True).tail(1000))
new_maxim['score'].value_counts()

C:\Users\onirutlA\AppData\Local\Temp\ipykernel_5140\2711205182.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_maxim = new_maxim.append(maxim.copy(deep=True).tail(1000))


1.0    1000
5.0    1000
Name: score, dtype: int64

In [ ]:
new_maxim.to_csv('{}maxim_1_5_2K_10.csv'.format(workspace_dir), quoting=csv.QUOTE_ALL)

### Filter 7 Words Sentence

In [ ]:
gojek = pd.read_csv(sentiment_gojek, quoting=csv.QUOTE_ALL, index_col='reviewId')
grab = pd.read_csv(sentiment_grab, quoting=csv.QUOTE_ALL, index_col='reviewId')
maxim = pd.read_csv(sentiment_maxim, quoting=csv.QUOTE_ALL, index_col='reviewId')

print(gojek.shape)
gojek['score'].value_counts()

(89770, 8)


5.0    73201
1.0    16569
Name: score, dtype: int64

In [ ]:
maxim['lowercased'] = maxim['content'].map(lowercase)
maxim['cleaned'] = maxim['lowercased'].map(text_cleaning)
maxim['slang_handled'] = maxim['cleaned'].map(slang_handling)
maxim['stopword_removed'] = maxim['slang_handled'].map(remove_stopword)
maxim['stemmed'] = maxim['stopword_removed'].map(stemming)
maxim['tokenized'] = maxim['stemmed'].map(tokenizing)
maxim = maxim[['content', 'lowercased', 'cleaned', 'slang_handled', 'stopword_removed', 'stemmed', 'tokenized', 'score']]
maxim['word_count'] = maxim['cleaned'].map(lambda x: x.split())
maxim

,content,lowercased,cleaned,slang_handled,stopword_removed,stemmed,tokenized,score,word_count
reviewId,,,,,,,,,
dc6878fe-f176-487e-a9e4-3e1a92c4fd26,Maps nya kocak alamat gaada semua susah amat n...,maps nya kocak alamat gaada semua susah amat n...,maps nya kocak alamat gaada semua susah amat n...,maps nya kocak alamat enggak ada semua susah a...,maps kocak alamat susah nyarinya ubah rute gil...,maps kocak alamat susah nyarinya ubah rute gil...,"[maps, kocak, alamat, susah, nyarinya, ubah, r...",1.0,"[maps, nya, kocak, alamat, gaada, semua, susah..."
4e4a51bb-206e-4ea0-8b40-007f69043907,"Saat saya melakukan order food, pencarian drai...","saat saya melakukan order food, pencarian drai...",saat saya melakukan order food pencarian drai...,saat saya melakukan order food pencarian draiv...,order food pencarian draivernya ketemu orderan...,order food cari draivernya ketemu order makan ...,"[order, food, cari, draivernya, ketemu, order,...",1.0,"[saat, saya, melakukan, order, food, pencarian..."
18f99180-ee86-496b-995d-f3f65596bc2d,Baru pesan aja udah minta tarip kaya angkot pa...,baru pesan aja udah minta tarip kaya angkot pa...,baru pesan aja udah minta tarip kaya angkot pa...,baru pesan saja sudah meminta tarip kayak angk...,pesan tarip kayak angkot aplikasi uang segitu,pesan tarip kayak angkot aplikasi uang segitu,"[pesan, tarip, kayak, angkot, aplikasi, uang, ...",1.0,"[baru, pesan, aja, udah, minta, tarip, kaya, a..."
af56172a-cc7b-48b6-ae77-4112fb510484,"Bagusnya di awal doang, setelah lebih lama pak...","bagusnya di awal doang, setelah lebih lama pak...",bagusnya di awal doang setelah lebih lama pak...,bagusnya di awal doang setelah lebih lama paka...,bagusnya doang pakai susah banget driver pakai...,bagus doang pakai susah banget driver pakai fi...,"[bagus, doang, pakai, susah, banget, driver, p...",1.0,"[bagusnya, di, awal, doang, setelah, lebih, la..."
4eae708d-d8b7-4bd4-accb-2cfdfd369c90,Aplikasi ini buruk sekali saya tidak ada mesan...,aplikasi ini buruk sekali saya tidak ada mesan...,aplikasi ini buruk sekali saya tidak ada mesan...,aplikasi ini buruk sekali saya tidak ada mesan...,aplikasi buruk mesan daftar kode verifikasi te...,aplikasi buruk mesan daftar kode verifikasi te...,"[aplikasi, buruk, mesan, daftar, kode, verifik...",1.0,"[aplikasi, ini, buruk, sekali, saya, tidak, ad..."
...,...,...,...,...,...,...,...,...,...
dc10c361-014d-4e30-b71e-de323beef663,Tolong diadakan promo biar makin ramai customerny,tolong diadakan promo biar makin ramai customerny,tolong diadakan promo biar makin ramai customerny,tolong diadakan promo biar makin ramai customerny,tolong diadakan promo biar ramai customerny,tolong ada promo biar ramai customerny,"[tolong, ada, promo, biar, ramai, customerny]",5.0,"[tolong, diadakan, promo, biar, makin, ramai, ..."
164a5c78-c3c3-4e56-881c-0e4778ef846c,Terima kasih sudah bantu angkat nenek saya,terima kasih sudah bantu angkat nenek saya,terima kasih sudah bantu angkat nenek saya,terima kasih sudah bantu angkat nenek saya,terima kasih bantu angkat nenek,terima kasih bantu angkat nenek,"[terima, kasih, bantu, angkat, nenek]",5.0,"[terima, kasih, sudah, bantu, angkat, nenek, s..."
5005c096-6dc0-461d-a2d4-33dc40e30b27,Tks semoga makin banyak armada dan pelayanan t...,tks semoga makin banyak armada dan pelayanan t...,tks semoga makin banyak armada dan pelayanan t...,tks semoga makin banyak armada dan pelayanan t...,tks semoga armada pelayanan bagus,tks moga armada layan bagus,"[tks, moga, armada, layan, bagus]",5.0,"[tks, semoga, makin, banyak, armada, dan, pela..."


In [ ]:
maxim = maxim[maxim['word_count'].str.len() >= 7]
maxim.to_csv('{}maxim_filter_7.csv'.format(workspace_dir), quoting=csv.QUOTE_ALL)

In [ ]:
maxim['word_count'].str.len().describe()

count    39456.000000
mean        16.741535
std         13.583175
min          7.000000
25%          8.000000
50%         12.000000
75%         19.000000
max        105.000000
Name: word_count, dtype: float64

In [ ]:
maxim['tokenized'].str.len().describe()

count    39456.000000
mean         9.248631
std          6.870806
min          0.000000
25%          5.000000
50%          7.000000
75%         11.000000
max         66.000000
Name: tokenized, dtype: float64

In [ ]:
print(maxim.shape)
maxim['score'].value_counts()

(39456, 9)


5.0    26074
1.0    13382
Name: score, dtype: int64

In [ ]:
new_maxim = maxim.copy(deep=True).head(1000)
new_maxim = new_maxim.append(maxim.copy(deep=True).tail(1000))
new_maxim['score'].value_counts()

C:\Users\onirutlA\AppData\Local\Temp\ipykernel_5140\2711205182.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_maxim = new_maxim.append(maxim.copy(deep=True).tail(1000))


1.0    1000
5.0    1000
Name: score, dtype: int64

In [ ]:
new_maxim.to_csv('{}maxim_1_5_2K_7.csv'.format(workspace_dir), quoting=csv.QUOTE_ALL)

# Merge Dataset

In [ ]:
dataset = new_gojek.append(new_grab)
dataset = dataset.append(new_maxim)
dataset

C:\Users\onirutlA\AppData\Local\Temp\ipykernel_5140\2244918956.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = new_gojek.append(new_grab)
C:\Users\onirutlA\AppData\Local\Temp\ipykernel_5140\2244918956.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dataset = dataset.append(new_maxim)


,content,lowercased,cleaned,slang_handled,stopword_removed,stemmed,tokenized,score,word_count
reviewId,,,,,,,,,
dc6878fe-f176-487e-a9e4-3e1a92c4fd26,Maps nya kocak alamat gaada semua susah amat n...,maps nya kocak alamat gaada semua susah amat n...,maps nya kocak alamat gaada semua susah amat n...,maps nya kocak alamat enggak ada semua susah a...,maps kocak alamat susah nyarinya ubah rute gil...,maps kocak alamat susah nyarinya ubah rute gil...,"[maps, kocak, alamat, susah, nyarinya, ubah, r...",1.0,"[maps, nya, kocak, alamat, gaada, semua, susah..."
4e4a51bb-206e-4ea0-8b40-007f69043907,"Saat saya melakukan order food, pencarian drai...","saat saya melakukan order food, pencarian drai...",saat saya melakukan order food pencarian drai...,saat saya melakukan order food pencarian draiv...,order food pencarian draivernya ketemu orderan...,order food cari draivernya ketemu order makan ...,"[order, food, cari, draivernya, ketemu, order,...",1.0,"[saat, saya, melakukan, order, food, pencarian..."
18f99180-ee86-496b-995d-f3f65596bc2d,Baru pesan aja udah minta tarip kaya angkot pa...,baru pesan aja udah minta tarip kaya angkot pa...,baru pesan aja udah minta tarip kaya angkot pa...,baru pesan saja sudah meminta tarip kayak angk...,pesan tarip kayak angkot aplikasi uang segitu,pesan tarip kayak angkot aplikasi uang segitu,"[pesan, tarip, kayak, angkot, aplikasi, uang, ...",1.0,"[baru, pesan, aja, udah, minta, tarip, kaya, a..."
af56172a-cc7b-48b6-ae77-4112fb510484,"Bagusnya di awal doang, setelah lebih lama pak...","bagusnya di awal doang, setelah lebih lama pak...",bagusnya di awal doang setelah lebih lama pak...,bagusnya di awal doang setelah lebih lama paka...,bagusnya doang pakai susah banget driver pakai...,bagus doang pakai susah banget driver pakai fi...,"[bagus, doang, pakai, susah, banget, driver, p...",1.0,"[bagusnya, di, awal, doang, setelah, lebih, la..."
4eae708d-d8b7-4bd4-accb-2cfdfd369c90,Aplikasi ini buruk sekali saya tidak ada mesan...,aplikasi ini buruk sekali saya tidak ada mesan...,aplikasi ini buruk sekali saya tidak ada mesan...,aplikasi ini buruk sekali saya tidak ada mesan...,aplikasi buruk mesan daftar kode verifikasi te...,aplikasi buruk mesan daftar kode verifikasi te...,"[aplikasi, buruk, mesan, daftar, kode, verifik...",1.0,"[aplikasi, ini, buruk, sekali, saya, tidak, ad..."
...,...,...,...,...,...,...,...,...,...
c226bfb6-7c97-4517-9833-2da3db0ba314,"tambah lagi kecepatan untuk loading data nya, ...","tambah lagi kecepatan untuk loading data nya, ...",tambah lagi kecepatan untuk loading data nya ...,tambah lagi kecepatan untuk loading data nya d...,kecepatan loading data membanyaki promo,cepat loading data banyak promo,"[cepat, loading, data, banyak, promo]",5.0,"[tambah, lagi, kecepatan, untuk, loading, data..."
dc10c361-014d-4e30-b71e-de323beef663,Tolong diadakan promo biar makin ramai customerny,tolong diadakan promo biar makin ramai customerny,tolong diadakan promo biar makin ramai customerny,tolong diadakan promo biar makin ramai customerny,tolong diadakan promo biar ramai customerny,tolong ada promo biar ramai customerny,"[tolong, ada, promo, biar, ramai, customerny]",5.0,"[tolong, diadakan, promo, biar, makin, ramai, ..."
164a5c78-c3c3-4e56-881c-0e4778ef846c,Terima kasih sudah bantu angkat nenek saya,terima kasih sudah bantu angkat nenek saya,terima kasih sudah bantu angkat nenek saya,terima kasih sudah bantu angkat nenek saya,terima kasih bantu angkat nenek,terima kasih bantu angkat nenek,"[terima, kasih, bantu, angkat, nenek]",5.0,"[terima, kasih, sudah, bantu, angkat, nenek, s..."


In [ ]:
dataset['score'].value_counts()

1.0    3000
5.0    3000
Name: score, dtype: int64

In [ ]:
dataset.to_csv('{}dataset_filtered_7.csv'.format(workspace_dir), quoting=csv.QUOTE_ALL)